# Text

There are lots of things you might want to do with documents
 - group them together
 - summarize one or several of them
 - classify them (e.g. sentiment, tagging)
 - explore, e.g. temporal trends in what they discuss

To do any of these things using a computer, you first need to convert words into numbers.

[https://github.com/fastforwardlabs/usaa-lda](https://github.com/fastforwardlabs/usaa-lda)

## Bags of words

The "bag of words" approach is the simplest method for this.

![Bag of words](img/bagwords.png)

There are at least three problems with this approach:

 - **long, sparse data**: a short text is turned into a very long series of numbers, most of which are zero ("sparse").
 
 - **synonyms and multiple meanings**: "movies" and "films" are different (they should not be), and "bow" (that shoots an arrow) and "bow" (you make to a King) are not (they should be)
 
 - **word order**: totally lost ("man bites dog" and "dog bites man" are the same).

We're going to look at a technique that addresses the first of these two problems.

# Topic modeling

Topic modeling is a statistical method to find groups of words that tend to co-occur in a corpus of documents.

For example, maybe the words "movie", "film" and "director" often occur in the same documents. That would make them a "topic".

Topic modeling algorithms find these groups automatically. They are an instance of the class of algorithms known as "unsupervised machine learning".

In doing this, we become able to express documents as a combination of a relatively small number (~100) of topics, rather than thousands of words (most of which don't occur), and we can treat documents about "films" and "movies" similarly.

# Topic modeling workflow

Topic modeling has two steps:

 - learn topics from a corpus of representative documents
 - figure out which of these topics occur in the particular document(s) you're interested in
 
If you're a machine learning person, you'll recognize these as training and evaluation.

<img src="img/lda_topics.png" width=50%>

<img src="img/lda_evaluate.png" width=50%>

Once you've done the second step, you've expressed your new document as a short vector of numbers that you can no do all sorts of things with:

 - group documents together
 - summarize one or several documents
 - classify it (e.g. sentiment, tagging)
 - explore, e.g. temporal trends in what people are discussing, e.g. [Time-series plots of 1000 topics extracted from 20 years of the New York Times.](http://christo.cs.umass.edu/NYT/)

## Latent Dirichlet Allocation

The best known and best algorithm for finding topics in a corpus is Latent Dirichlet Allocation. It's got a complicated name and, to be frank, it's a complicated algorithm. If you'd like to begin to dig into the details, there are two resources I recommend very highly!

 - [Tim Hopper's PyData NYC 2015 talk](https://www.youtube.com/watch?v=_R66X_udxZQ)
 - [David Blei's ACM article](https://www.cs.princeton.edu/~blei/papers/Blei2012.pdf)
 
For the purposes of this talk, I'm just going to say it finds groups of words that co-occur by magic!

The good news is, there are several excellent open source implementations of the algorithm, and we're going to use one of those today.

We're going to apply it to a public dataset of Amazon product reviews.

# Load data

The cell below opens the file and loads it into a pandas dataframe.

There's a lot going on in this line, all of which is useful to understand if you're a Python programmer, but none of which is necessary to understand if you're only interested in LDA.

If you would like a more detailed explanation of what's going on, please see [the Data notebook](data.ipynb)

In [34]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import json
import numpy as np
import pandas as pd

with open("reviews.json", 'rb') as f:
    reviews = pd.DataFrame(json.loads(l.decode()) for l in f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


A pandas dataframe is a structured table-like object that, among many other things, supports a bunch of SQL-like operations and handles fiddly data types like data and times well. I don't know much about R, but I understand R has objects like this too.

`head` allows us to see the first few rows:

In [35]:
reviews.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1223000893,"[0, 0]",3.0,I purchased the Trilogy with hoping my two cat...,"01 12, 2011",A14CK12J7C7JRK,Consumer in NorCal,Nice Distraction for my cats for about 15 minutes,1294790400
1,1223000893,"[0, 0]",5.0,There are usually one or more of my cats watch...,"09 14, 2013",A39QHP5WLON5HV,Melodee Placial,Entertaining for my cats,1379116800
2,1223000893,"[0, 0]",4.0,I bought the triliogy and have tested out all ...,"12 19, 2012",A2CR37UY3VR7BN,Michelle Ashbery,Entertaining,1355875200
3,1223000893,"[2, 2]",4.0,My female kitty could care less about these vi...,"05 12, 2011",A2A4COGL9VW2HY,Michelle P,Happy to have them,1305158400
4,1223000893,"[6, 7]",3.0,"If I had gotten just volume two, I would have ...","03 5, 2012",A2UBQA85NIGLHA,"Tim Isenhour ""Timbo""",You really only need vol 2,1330905600


Individual columns can be accessed as keys:

In [36]:
reviews['asin'].head()

0    1223000893
1    1223000893
2    1223000893
3    1223000893
4    1223000893
Name: asin, dtype: object

In [37]:
print("{} reviews".format(len(reviews)))
print("of {} products ".format(len(reviews.asin.unique())))
print("by {} unique authors ".format(len(reviews.reviewerID.unique())))

150000 reviews
of 8019 products 
by 19834 unique authors 


In [38]:
texts = reviews.loc[:19999, 'reviewText']

In [39]:
print(texts[0])

I purchased the Trilogy with hoping my two cats, age 3 and 5 would be interested.  The 3 yr old cat was fascinated for about 15 minutes but when the same pictures came on, she got bored.  The 5 year old watched for about a few minutes but then walked away. It is possible that because we have a wonderful courtyard full of greenery and trees and one of my neighbors has a bird feeder, that there is enough going on outside that they prefer real life versus a taped version.  I will more than likely pass this on to a friend who has cats that don't have as much wildlife to watch as mine do.


# Vectorize reviews

You have to preprocess the text a little before you apply LDA: you need to split documents into words, and you need to turn words into vectorized numbers.

Ironically, in order to get the benefits of LDA, we first need to run bag of words on our data!

The code to do this is built into scikit-learn.

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
X = vectorizer.fit_transform(texts)

In [41]:
X.shape

(20000, 10000)

# Learn topics using LDA

In [42]:
from sklearn.decomposition import LatentDirichletAllocation

In [43]:
%%time
lda = LatentDirichletAllocation(n_topics=100, learning_method='batch', n_jobs=-2, random_state=0)
lda.fit(X)

CPU times: user 2.17 s, sys: 3.61 s, total: 5.77 s
Wall time: 1min 35s


# Inspect topics

The first thing you should do when you fit a topic model is inspect a few of the words that dominate each topic to check that the topics are coherent.

To do this, we need to look at the `components_` attribute now attached to `lda`. This is an array with `n_topics` rows and a number of columns equal to the size of the vocabulary.

In [44]:
print(lda.components_.shape)

(100, 10000)


Each number in this array is the weight of the corresponding word in the corresponding topic.

The weights of each topic should add up to one, i.e. each row of `lda.components_` should add up to 1.

In [45]:
lda.components_.sum(axis=1)

array([  4433.27370724,   6655.65363106,   4417.59636293,  11832.90931634,
         3963.1514794 ,  10722.5952035 ,   5332.21591946,   3246.91103429,
         4706.29046561,  18583.9966215 ,   3390.96693158,   6741.89190383,
         4317.35630515,   8728.07765604,   3733.66742834,  19060.45652928,
         3249.61949682,   7057.45953921,   3118.25421223,   3886.37900669,
         3014.01380836,   3640.80612269,   3879.98580666,  10330.34614701,
         3393.53451897,   3318.43067221,   5612.35654271,   3719.26391037,
         3191.7666571 ,  12937.90440418,   4010.92549452,   4817.59366305,
        14707.85052824,   3811.6920899 ,   6638.30226746,   8799.7396821 ,
         4645.7271835 ,   5094.27986301,  17669.6498383 ,   4931.202064  ,
         3790.18781725,   6694.45468661,  30470.04926392,   2625.0278297 ,
         4988.13237731,   3597.88434063,   3692.79018628,   2972.06107775,
         7199.7830463 ,   5628.91583097,   3349.75583582,  13269.54699479,
         9543.436802  ,  

Oh dear! It turns out there's a bug in scikit-learn's implementation of LDA. Let's fix it here. This should be fixed in the next version of scikit-learn.

In [46]:
lda.components_ /= lda.components_.sum(axis=1)[:, None]

This array of topics and words (or terms) is usually called the topic-term matrix, so let's save it under that name:

In [47]:
topic_term = lda.components_

The word corresponding to each column in this array, which we'll call the `vocabulary`, is available as a list from `vectorizer.get_feature_names()`

In [48]:
vocabulary = vectorizer.get_feature_names()
print(vocabulary[:10000:1000]) # print the 0th, 999th, 1999th, 2999th, etc. item in the vocabulary

['00', 'bettas', 'contemplating', 'element', 'grunter', 'lays', 'nutro', 'ranchus', 'skinnier', 'tipped']


Now let's look at the top 10 words that dominate each topic.

It does that by going through each row (i.e. each topic) in the `topic_term` array, finding the biggest numbers in that row, then finding the corresponding word.

In [49]:
def print_topic(topic_term, topic_id, vocabulary):
    print("{:2d} ".format(topic_id) + " ".join(vocabulary[i] for i in topic_term[i].argsort()[:-11:-1]))

for i, _  in enumerate(topic_term):
    print_topic(topic_term, i, vocabulary)

 0 large small star just think size dog took old kitten
 1 bed warm heat night cold pad winter hot hours use
 2 product continue use save using koi year old years better
 3 litter box clean cat use just pan like don cats
 4 just product junk works great piece snails good make using
 5 food feeder time feeding day cat feed timer just set
 6 air bubbles tubing ear ears just works like pump airline
 7 couch long hip energy lasting adorable berries great love summer
 8 buy product package great buying disappointed colors good cost im
 9 filter filters tank media aquaclear water use canister sponge filtration
10 pups dogs love ones cubes make favorite like nylabones ve
11 quick use easy great clippers work clean little grip smooth
12 dog just bear like milk teddy say really rex little
13 ball dog hard treat handle like plastic roll kibble hole
14 product did saved away crickets date rid parrot starts rats
15 water fish test tank use ammonia kit ph product aquarium
16 snake refill don just w

In this case, the topics are reasonably coherent, so I'm going to move on.

If things look messy:
 - n_topics might be too large, either for the diversity in the corpus (maybe there really aren't 1000 topics), or for the number of documents you have (you just don't have enough data)
 - n_topics might be too low (real topics have to be merged together by the algorithm, which doesn't work well)
 - you've got a bug!
 
Setting n_topics very small (say 5) or very high (say 1000) is a good way of building up some intuition for what works (although beware `n_topics=1000` will take a long time to run).

# Inspect a document in the corpus

The topic model is the lens through which we're going to view future documents.

But let's first look at our existing documents through this lens.

To do that we have to transform the documents we trained on to be distributions of topics (e.g. document 1 is 20% topic A, 30% topic B, etc.)

We do that by running the `lda.transform` method on the vectorized documents `X`:

In [50]:
doc_topic = lda.transform(X)

`doc_topic` has a row for each document, and a column for each topic.

In [51]:
doc_topic.shape

(20000, 100)

Again, each row should add up to 1, and again they don't because of a bug in scikit-learn, so let's fix that:

In [52]:
doc_topic /= doc_topic.sum(axis=1)[:, None]

Finally let's look at the topic distribution of a random document

In [53]:
reviews.loc[8, 'reviewText']

"Works great and dog doesn't hate the taste.  Gum health is important so just have to brush those pearly whites."

In [54]:
top_topics = (doc_topic[7]).argsort()[:-6:-1]
print(top_topics)

[34 27 98 17 51]


What are these topics?

In [55]:
for i in top_topics:
    print_topic(topic_term, i, vocabulary)

34 teeth dog dogs toothpaste brush taste flavor breath like loves
27 34 dog just right received use does way stuff used
98 poop grass scooper rake just like sticks works poo ve
17 smell dog smells dry like shampoo skin bath use good
51 toy loves keeps dog puppy play chew busy great kong


# Visualization

pyLDAvis is a comprehensive package for visualizing the results of a topic model. It's useful for understanding the structure of the model you've just discovered. The topics exist in a huge space. This package squeezes things down to 2D so we can look at it on the screen.

In my experience, it generates a ton of spurious warnings, so let's disable warnings for this package when we import it (a useful trick!)

In [56]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    try:
        import pyLDAvis
    except ImportError:
        print('ERROR: pyLDAvis not installed! Skip to next section!')

In addition to the `topic_term` and `doc_topic` matrices, pyLDAvis needs to know how often each word occurs in the entire corpus, and how long each document is. Here are calculations that give those.

In [57]:
term_frequency = np.asarray(X.sum(axis=0)).squeeze()
doc_lengths = [len(t) for t in texts]

In [58]:
lda_vis = pyLDAvis.prepare(topic_term_dists=topic_term,
                           doc_topic_dists=doc_topic,
                           doc_lengths=doc_lengths,
                           vocab=vocabulary,
                           term_frequency=term_frequency)

In [59]:
pyLDAvis.display(lda_vis)

# Put all this together in a Pipeline and persist the model

The process of getting from document to topic distribution is a little fiddly. We need to:
 - Vectorize the document (using the same vocabulary we used when training above)
 - Transform the document using the LDA object
 
scikit-learn allows us to bundle these steps (and more!) together in an object called a `Pipeline`, which we can save to disk, reload, and work with again. Let's build one, train it, and save it.

**WARNING**: this next cell will take a while to execute the first time you run it. After that though, the model will be loaded from disk.

In [60]:
import pickle
from sklearn.pipeline import make_pipeline

try:
    with open('topic_model.pkl', 'rb') as f:
        topic_pipeline = pickle.load(f)
    pipeline_vocabulary = topic_pipeline.steps[0][1].get_feature_names()
except IOError:
    topic_pipeline = make_pipeline(
        CountVectorizer(stop_words='english', max_features=10000),
        LatentDirichletAllocation(n_topics=100, learning_method='batch', n_jobs=-2, random_state=0)
    )
    topic_pipeline.fit(texts)
    with open('topic_model.pkl', 'wb') as f:
        pickle.dump(topic_pipeline, f)
        
pipeline_vocabulary = topic_pipeline.steps[0][1].get_feature_names()
pipeline_topic_term = topic_pipeline.steps[1][1].components_

# Determine topics of a new document

The single document we looks at above was a pretty short document. Let's make a more interesting, longer document out of all the reviews of that product.

In [61]:
randomreviews = " ".join(texts[1100:1105])

In [62]:
print(randomreviews)

Careful people! Even though this chew toy says for powerful chewers, it also says for dogs up to 50 lbs in VERY tiny print on the bottom of the package. I gave it to my 79 lb Lab Mix puppy anyway, since I had already bought it... and lo and behold, it began falling apart in only a few moments.After removing the toy and picking up the white bits all over... I thought to come here and warn people. I find it odd how it's for powerful chewers yet only includes medium sized dogs in the toy description! With pit bull terriers and a few other smaller breeds as an exception... usually smaller dog means less destructive chewing ability. Senseless, really.If you have a dog that falls under this weight category... except for a breed with very powerful jaws, this toy probably suits just fine, but if you have a large dog, don't even think about it. I paid almost $9 for a toy that begun to disintegrate within a few seconds. I bought the Nylabone dinosaur chew toy for my 80lb. Greyhound and instead m

In [63]:
doc_topic = topic_pipeline.transform([randomreviews])

In [64]:
print(doc_topic.shape)

(1, 100)


In [65]:
top_topics = (doc_topic[0]).argsort()[:-6:-1]
print(top_topics)

[65 31 70 53 86]


In [66]:
for i in top_topics:
    print_topic(pipeline_topic_term, i, pipeline_vocabulary)

65 chew dog toy chewer chewing toys dogs minutes chewed bone
31 chew nylabone puppy dog dogs loves chews chewing like bones
70 large don dog ends like size just small tend medium
53 hair brush loose dog fur cat doesn use like does
86 toy favorite loves dog toys cat love play cats plays


# What next?

Use the `doc_topic` array for a downstream task, e.g.
 - corpus exploration (remember the [NYT visualization](http://christo.cs.umass.edu/NYT/))
 - document clustering, e.g. use something like `KMeans` (in scikit-learn) to visualize which documents are most similar in terms of their topics, which may surface groups of topics or groups of documents

## Summarization

Here's a short algorithm, but see [Fast Forward Labs Report 04](http://ff04.fastforwardlabs.com) for details:
  - Train LDA on all products of a certain type (e.g. all the books)
  - Treat all the reviews of a particular product as one document, and infer their topic distribution
  - Infer the topic distribution for each sentence
  - For each topic that dominates the reviews of a product, pick some sentences that are themselves dominated by that topic

<img src="img/strain.png">

Be aware of limitations:
 - Choosing `n_topics` is an art rather than a science!
 - The topics don't come with names. Sometimes they overlap. Sometimes they're not what you want them to be. For example, if you run a topic model on the NYT corpus, there's no guarantee you'll get topics that correspond to the sections of the newspaper (business, metro, world, sport, etc.!)

The way you used `fit` and `transform` for both the `vectorizer`, `lda`, and `topic_pipeline` objects is generic across scikit-learn, so play with scikit-learn, e.g. [Andreas Mueller's presentation](https://www.youtube.com/watch?v=8CzwlZbwDkI) is a good place to start.
 
Remember if you're interested in the LDA algorithm itself, take a look at

 - [Tim Hopper's PyData NYC 2015 talk](https://www.youtube.com/watch?v=_R66X_udxZQ)
 - [David Blei's ACM article](https://www.cs.princeton.edu/~blei/papers/Blei2012.pdf)